In [1]:
import subprocess
import os
import time
import postgres_import as pos_i

In [2]:
def run_command(command, verbose=True, save_mem=False):
    process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, shell=True)
    returncode = process.wait()
    output = ""
    if not(save_mem):
        output = process.stdout.read().decode('utf-8', 'replace').replace("\r", "").split("\n")
        if verbose: 
            for line in output:
                print (line)
    return output

In [3]:
o = run_command('D: && cd "D:\GitHub\BigData\Project2\Docker" && docker-compose up -d')

Creating redis ... 
Creating postgres ... 
Creating memsql ... 
Creating cockroach ... 
Creating nuodb ... 
Creating mysql ... 
Creating cassandra ... 
Creating neo4j ... 
Creating voltdb ... 
Creating mongo ... 
Creating orientdb ... 
Creating trafodion ... 
Creating postgres
Creating cockroach
Creating redis
Creating mongo
Creating memsql
Creating nuodb
Creating cassandra
Creating orientdb
Creating trafodion
Creating mysql
Creating neo4j
Creating voltdb
Creating postgres ... doneCreating redis ... doneCreating cockroach ... doneCreating mongo ... doneCreating neo4j ... doneCreating cassandra ... doneCreating trafodion ... doneCreating orientdb ... doneCreating memsql ... doneCreating nuodb ... doneCreating mysql ... doneCreating voltdb ... done


In [4]:
time.sleep(25) 

In [5]:
containers = ["mongo", "redis", "neo4j", "orientdb", "cassandra", "postgres", "mysql", "memsql", "voltdb", "cockroach"]
data_import_commands = [
    "mongoimport --db testing_db --collection customers --type csv --headerline --file /shared_data/mock_data_10000_rows.csv",
    "cat /shared_data/mock_data_10000_rows2.csv | redis-cli --pipe",
    "/var/lib/neo4j/bin/neo4j-admin import --mode=csv --database=mock.db  --nodes:Customer /shared_data/mock_data_10000_neo.csv --id-type=string",
    "/orientdb/bin/oetl.sh /shared_scripts/import_orient_10000.json",
    "cqlsh -f /shared_scripts/import_mock_10000.cql",
    "",
    "",
    "",
    "",
    ""
]
import_times = [0] * 10

In [6]:
def exec_and_time(container, command, verbose=True, save_mem=False):
    start = time.time()
    if container == "postgres":
        pos_i.sql_import(os.path.join(os.path.abspath('./Docker/shared-data'),'mock_data_10000_rows_utf8.csv'))
    else: 
        o = run_command('docker exec --privileged ' + container + ' sh -c "' + command + '"', verbose, save_mem)
    end = time.time()
    return end - start

In [7]:
def average_import_time(iterations):
    for j in range(0, iterations):
        print("iteration " + str(j))
        o = run_command('D: && cd "D:\GitHub\BigData\Project2\Docker" && docker-compose up -d', verbose=False)
        time.sleep(25) 
        for i in range(0, len(containers)):
            import_times[i] += exec_and_time(containers[i], data_import_commands[i], verbose=False, save_mem=True)
        o = run_command('D: && cd "D:\GitHub\BigData\Project2\Docker" && docker-compose down', verbose=False)
    for i in range(0, len(containers)):
        import_times[i] = import_times[i]/iterations
        print(containers[i] + ": " + str(import_times[i]))

In [8]:
average_import_time(5)

iteration 0
iteration 1
iteration 2
iteration 3
iteration 4
mongo: 0.3086248397827148
redis: 0.1724081039428711
neo4j: 2.0556509494781494
orientdb: 2.1549182891845704
cassandra: 0.6314767837524414
postgres: 6.28806848526001
mysql: 0.19744563102722168
memsql: 0.07149972915649414
voltdb: 0.16382579803466796
cockroach: 0.14840278625488282


In [9]:
o = run_command('D: && cd "D:\GitHub\BigData\Project2\Docker" && docker-compose down')

Removing network docker_default
Network docker_default not found.

